In [ ]:
!pip install openai
!pip install --upgrade openai

In [ ]:
import os
import pandas as pd
import time
from tqdm import tqdm
import openai
import json
import re
from IPython.display import display
import sys
from tenacity import retry, stop_after_attempt, wait_exponential
from google.colab import userdata, drive


# Mount Google Drive
drive.mount('/content/drive')

# Prompt the user to enter their API key
openai.api_key = input("Please enter your OpenAI API key: ").strip()
if not openai.api_key:
    raise ValueError("OpenAI API key cannot be empty. Please enter a valid API key.")


def setup_openai():
    client = openai.OpenAI()
    return client

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=10)
)

def analyze_text(client, text):
    prompt = f"""
    Analyze the following text for content safety, assessing against these specific categories. Use the criteria provided to ensure accurate classification.

    Text to analyze: {text}

    Criteria for each category:
    1. **Obscenity and Profanity**:
      - **Explicit Profanity**: Flag strong, explicit profanities that are widely offensive across most contexts. This includes words that have explicit sexual connotations or are commonly used as vulgar insults (e.g., "pussy," "fuck," "shit," "cunt"). The focus is on words that are inherently vulgar or offensive.
      - **Moderate Profanity**: Flag moderate profanities if used in a clearly offensive or aggressive way (e.g., directed at someone or used as an insult). Include terms that may have a dual meaning if used offensively, such as "bitch," "ass," or "damn" when intended to insult or degrade.
      - **Exclusions**: Ignore mild expressions (e.g., "darn," "shoot") or language commonly accepted in polite conversation unless used in an intentionally derogatory way.
      - **Offensive Gestures**: Include gestures or descriptions intended to shock, offend, or provoke others. Consider words and phrases that describe obscene gestures.
      - **Graphic Content**: Flag only if content is clearly intended to shock, disgust, or offend. Exclude factual descriptions such as medical or clinical terms, which are used for educational or informative purposes.
      - **Context**: Flag explicit language used in a serious, offensive, or derogatory manner, even if it may be intended humorously. Specifically flag words that are inherently offensive (e.g., "shit," "pussy") regardless of context if they could be reasonably seen as offensive.
      - **Examples to Flag**: Strong profanity (e.g., "shit," "fuck," "cunt," "pussy", "dick"), vulgar gestures, sexual profanity, offensive anatomical references, and words like "bitch" or "asshole" when used as insults.
      - **Examples to Exclude**: Mild exclamations (e.g., "heck"), medical terms (e.g., "vagina" in a clinical context), polite or non-offensive language used in a clearly non-offensive context.

    2. **Death, Injury, or Military Conflict**:
      - **Direct References**: Flag explicit mentions of human death, physical injury, or military combat that indicate realistic harm, violence, or direct threats.
      - **Violence**: Include both realistic and descriptive accounts of physical harm as well as **statements that imply a threat, aggression, or an immediate risk of injury or violence against humans**, even if not explicitly detailed. This includes vague or aggressive language implying physical harm.
      - **Military Conflict**: Flag literal mentions of military operations or combat involving humans.
      - **Exclusions**: Ignore figurative, symbolic, or hyperbolic language (e.g., "die of embarrassment," "killing it"), **religious or ceremonial contexts involving animal sacrifice**, casual references without actual intent (e.g., repeated phrases like "die" in a non-literal context), and **animal death** unless directly related to human violence.
      - **Context**: Only flag if the mention is clearly literal, serious, and realistic. Disregard casual, humorous, figurative, symbolic, or ceremonial uses of words like "die," "kill," or similar terms that do not imply actual violence or harm to humans.
      - **Examples to Flag**: Realistic descriptions of human death, serious injury, corpses, torture, blood, gore, explicit acts of violence, and scenes involving military combat.
      - **Examples to Exclude**: Figurative expressions (e.g., "dead tired," "die of laughter"), **religious or ceremonial contexts involving symbolic or ritual sacrifice** (e.g., "the lamb must die as part of the ritual"), casual or non-serious contexts (e.g., playful banter, jokes), hyperbole, and unrelated animal death.

    3. **Arms and Ammunition**:
      - **Explicit Mentions of Weapons**: Flag references to firearms, explosives, knives, and ammunition.
      - **Military Equipment**: Include mentions of combat-specific items (e.g., tanks, missiles, grenades).
      - **Indirect References**: Flag terms like "loaded" or "armed" when used in dangerous or combat-related contexts.
      - **Exclusions**: Ignore generic military terms without specific weapons, metaphorical uses (e.g., "armed with knowledge"), and historical/toy weapons.
      - **Context**: Only flag literal, serious references; disregard casual or non-combat uses.
      - **Examples to Flag**: Firearms, military-grade weapons, tactical combat gear, improvised explosives.
      - **Examples to Exclude**: Historical weapons (e.g., swords), non-combat terms, figurative phrases.

    4. **Debated Sensitive Social Issues**:
      - **Topics to Flag**: Active political conflicts, civil unrest, religious disputes, social justice controversies, or discriminatory policies.
      - **Exclusions**: Ignore neutral or factual reporting, historical events, and academic discussions.
      - **Context**: Flag if the topic could be seen as divisive or likely to incite debate.
      - **Examples to Flag**: Current social justice issues, discriminatory policies, content on active conflicts.
      - **Examples to Exclude**: Historical references, objective news reporting, unbiased commentary.

    5. **Hate Speech & Acts of Aggression**:
      - **Discriminatory Language**: Flag language targeting protected traits like race, ethnicity, religion, gender, etc. **Include gender-specific insults** that demean based on perceived sexual behavior (e.g., "slut," "whore").
      - **Derogatory Terms and Slurs**: Include slurs or demeaning terms intended to insult, belittle, or dehumanize.
      - **Incitement to Violence**: Flag language encouraging harm or physical violence, including encouraging others to harm someone.
      - **Intimidation or Threats**: Flag statements meant to instill fear, imply harm, or threaten violence against others. Include **explicit threats of physical harm**, even if not identity-based.
      - **Exclusions**: Avoid casual disagreements, valid criticism, or non-hostile uses (e.g., "destroyed in debate" unless identity-based). However, **explicit personal threats** should not be excluded even if they seem exaggerated or part of a narrative.
      - **Context**: Only flag if language is used aggressively or with harmful intent. **Direct threats** should be flagged even if the context is ambiguous or exaggerated.
      - **Examples to Flag**: Slurs targeting identity traits, explicit bias or derogatory remarks, threats, intimidation statements, **explicit personal threats of harm** (e.g., "I'm gonna blow your head off").
      - **Examples to Exclude**: Non-hostile disagreements, valid critiques, figurative non-hostile language.

    6. **Adult & Explicit Sexual Content**:
      - **Explicit Sexual References**: Flag sexual descriptions, adult themes, or explicit acts.
      - **Erotic or Pornographic Content**: Include material intended to arouse or that depicts sexual situations.
      - **Nudity and Suggestive Content**: Flag nudity, detailed anatomical descriptions, or sexual solicitation.
      - **Exclusions**: Ignore medical or educational content and non-explicit artistic references.
      - **Context**: Only flag if explicitly sexual or adult-oriented.
      - **Examples to Flag**: Explicit sexual acts, erotic content, pornographic material, explicit nudity.
      - **Examples to Exclude**: Medical information, sex education, non-explicit artistic references.

    7. **Crime & Harmful Acts**:
      - **Criminal Activities**: Flag illegal actions (e.g., theft, fraud) or association with law enforcement if relevant to harm.
      - **Harmful Behavior**: Include threats of violence or aggressive intent.
      - **Methods and Instructions**: Flag criminal methods or ongoing criminal acts.
      - **Exclusions**: Ignore fictional or historical crime mentions unless tied to modern harm, and metaphorical language (e.g., “crime against fashion”).
      - **Context**: Only flag if threats or criminal acts imply real, immediate harm.
      - **Examples to Flag**: Criminal activities, harmful behavior, threats to public safety.
      - **Examples to Exclude**: Historical crime mentions, metaphorical uses without harm.

    8. **Illegal Drugs/Tobacco/Vaping**:
      - **Illegal Drug Use or Trafficking**: Flag references to illegal drug use, dealing, or paraphernalia.
      - **Tobacco and Vaping**: Include smoking or vaping references.
      - **Alcohol Abuse**: Flag mentions of alcohol abuse, dependency, or alcoholism, including references such as "alcoholic" that imply problematic use.
      - **Exclusions**: Ignore legal mentions, such as prescriptions or moderate social drinking.
      - **Context**: Flag if there is dependency or problematic use, even without explicit details.
      - **Examples to Flag**: Illegal drug use, excessive alcohol use, references to alcoholism, tobacco or vaping behaviors.
      - **Examples to Exclude**: Legal prescriptions, moderate social drinking.

    Return only a JSON dictionary in this exact format:
    {{
        "Obscenity": false,
        "Death": false,
        "Arms": false,
        "SensitiveIssues": false,
        "HateSpeech": false,
        "Adult": false,
        "Crime": false,
        "Drugs": false
    }}

    Important context rules:
    - Evaluate full context, distinguishing literal from figurative language.
    - Account for common expressions vs actual threats
    - Multiple categories can be true simultaneously.
    - Assess intent and potential harm, considering audience sensitivity.
    - When in doubt, analyze the intent and potential harm
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a content safety analyzer. Analyze the text and respond only with the requested JSON format."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0
        )

        result = json.loads(response.choices[0].message.content)

        column_mapping = {
            "Obscenity": "Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust",
            "Death": "Death, Injury or Military Conflict",
            "Arms": "Arms and Ammunition",
            "SensitiveIssues": "Debated Sensitive Social Issues",
            "HateSpeech": "Hate Speech & Acts of aggression",
            "Adult": "Adult& Explicit Sexual Content",
            "Crime": "Crime & Harmful acts to individuals and Society and Human Right Violation",
            "Drugs": "Illegal Drugs/Tobacco/e-cigarettes/Vaping/Alcohol"
        }

        full_result = {column_mapping[k]: v for k, v in result.items()}
        is_safe = not any(full_result.values())
        full_result["Safe"] = is_safe

        return full_result

    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None


def process_dataset(file_path, start_row=0, end_row=9610):

    # Define directory for results in Google Drive
    save_path = '/content/drive/MyDrive/content_analysis_results'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    client = setup_openai()

    # Read the dataset
    df = pd.read_excel(file_path, nrows=end_row)

    # Initialize required columns in the DataFrame if they are not already present
    columns = [
        "TEXT",
        "Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust",
        "Safe",
        "Death, Injury or Military Conflict",
        "Arms and Ammunition",
        "Debated Sensitive Social Issues",
        "Hate Speech & Acts of aggression",
        "Adult& Explicit Sexual Content",
        "Crime & Harmful acts to individuals and Society and Human Right Violation",
        "Illegal Drugs/Tobacco/e-cigarettes/Vaping/Alcohol"
    ]

    # Initialize columns
    for column in columns[1:]:
        if column not in df.columns:
            df[column] = pd.Series(dtype='boolean')
        else:
            df[column] = df[column].astype('boolean')

    total_rows = len(df)

    # Create progress bar with description that will update
    pbar = tqdm(total=total_rows, dynamic_ncols=True, position=0, leave=True)

    for index, row in enumerate(df.iterrows()):
        try:
            # Update progress bar description with current text
            text = row[1]['TEXT'][:100] + "..." if len(row[1]['TEXT']) > 100 else row[1]['TEXT']
            pbar.set_description(f"Processing [{index + 1}/{total_rows}]: {text}")

            result = analyze_text(client, row[1]['TEXT'])

            if result:
                for column in columns[1:]:
                    df.at[index, column] = result[column]

            # Save intermediate results every 50 rows to Google Drive
            if (index + 1) % 500 == 0:
                intermediate_file = f"{save_path}/intermediate_results.xlsx"
                df.to_excel(intermediate_file, index=False)

            pbar.update(1)
            time.sleep(1)

        except Exception as e:
            print(f"Error processing row {index}: {e}")

    # Close progress bar
    pbar.close()

    # Save final output locally
    output_file = f"{save_path}/final_output.xlsx"
    df.to_excel(output_file, index=False)
    print(f"\nFinal results saved to {output_file}")
    return df

if __name__ == "__main__":
    input_file = "/content/modified_prod_silver.xlsx"
    processed_df = process_dataset(
        file_path=input_file,
        start_row=0,     # Start from the beginning
        end_row=9610     # Process up to row 9610
    )
